# PyTorch Basics

## Introduction to PyTorch

### What is PyTorch?

[PyTorch](https://pytorch.org/get-started/locally/) is a popular open-source deep learning framework known for its flexibility and ease of use. It's widely adopted in both research and industry for tasks ranging from simple machine learning models to complex neural networks.

### Why PyTorch?

- Dynamic computation graph: PyTorch's ability to dynamically build the computation graph at runtime makes it intuitive and easy to debug.
- Strong community support and integration with Python: PyTorch is Pythonic and integrates well with the Python data science stack.
- GPU acceleration: PyTorch makes it easy to move tensors to and from GPUs (supports Apple's Metal and Nvidia GPUs), which is crucial for training large models efficiently.

## Tensors in PyTorch

### What is a Tensor?

Tensors are the fundamental data structures in PyTorch, similar to NumPy arrays but with the added capability of being used on a GPU.

### Creating Tensors

In [1]:
%pip install -q torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

# Creating a tensor from a list
tensor_a = torch.tensor([1.0, 2.0, 3.0])
print(tensor_a)

# Creating a tensor with random values
tensor_b = torch.rand((2, 3))  # A 2x3 matrix of random numbers
print(tensor_b)

# Creating a tensor with zeros
tensor_c = torch.zeros((3, 3))  # A 3x3 matrix of zeros
print(tensor_c)


tensor([1., 2., 3.])
tensor([[0.6560, 0.0980, 0.8618],
        [0.1586, 0.7411, 0.3848]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


### Basic Tensor Operators

In [3]:
# Reshaping a tensor
tensor_reshaped = tensor_b.view(3, 2)  # Reshape to 3x2
print(tensor_reshaped)

# Tensor addition
tensor_sum = tensor_a + tensor_a
print(tensor_sum)

# Indexing
print(tensor_a[1])  # Access the second element


tensor([[0.6560, 0.0980],
        [0.8618, 0.1586],
        [0.7411, 0.3848]])
tensor([2., 4., 6.])
tensor(2.)


In [4]:
# Moving a tensor to GPU

# Check which device is available
available_device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Available device: {available_device}")

tensor_a_gpu = tensor_a.to(available_device)
print(tensor_a_gpu)

Available device: cpu
tensor([1., 2., 3.])


/home/eviofekeze/anaconda3/envs/snowEx/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Autograd: Automatic Differentiation

- PyTorch’s autograd system automatically calculates gradients, which are essential for training neural networks.
- Every operation on tensors keeps track of the computation history, allowing PyTorch to backpropagate errors automatically.

In [5]:
# Create a tensor with gradient tracking enabled
x = torch.tensor(2.0, requires_grad=True)

# Perform a computation
y = x ** 2 + 2* x ** 3

# Backpropagate to compute the gradient
y.backward()

# Print the gradient
print(x.grad)  # Should output 28.0, the derivative of x^2 + 2x^3 at x=2

tensor(28.)


$$y = x^2 + 2x^3$$
$$y = 2x + 6x^2$$
$$y = 2(2) + 6 \times(2)^2$$
$$y = 4 + (6 \times 4)$$
$$y = 4 + 2 $$
$$y = 28 $$

```{important}
This example shows how PyTorch automatically calculates the gradient of a tensor operation, which is essential for updating the weights during training.
```

## Dataset and DataLoaders

### What is a Dataset in PyTorch?

* Purpose: The `Dataset` class in PyTorch serves as an abstraction that allows you to manage, preprocess, and access your data in a consistent way.

* Key Features:
    - Handles how data is stored and accessed.
    - Allows for custom data transformations and preprocessing.
    - Integrates seamlessly with PyTorch’s `DataLoader` for efficient batching and shuffling.

### What is a DataLoader in PyTorch?

* Purpose: The `DataLoader` is an iterable that abstracts the complexity of handling data in batches, shuffling, and parallel loading.

* Key Features:
    - Efficiently loads data in mini-batches during training.
    - Automatically shuffles data at the start of each epoch (if specified).
    - Supports parallel data loading using multiple workers.


In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        # Retrieve the data sample and label at the specified index
        sample = self.data[idx]
        target = self.targets[idx]
        return sample, target

In [ ]:
from dataclasses import dataclass, field
from typing import Any
import torch
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
import utils.config as config


@dataclass(kw_only=True)
class InfrasoundDataset(Dataset):
    """
    Class to create Infrasound dataset from audio file + annotation file
    The Class applies the following functions
    1. A Transformation specified in the configuration file, This can either be MFCC, Mel-Spectrgram or any added
        in the future
    2. A function to mix down audio to mono channel if more than on channel
    3. A function to resample data to target sample rate specified in the config file
    4. A function to pad the audio if sample is less that target sample
    5. A function to trim sample

    Args:
        labels_file: path to file containing labels
        waveform_file: path to file containing audio signals
        transformation: features to be extracted from signal
        target_sample_rate: the sample rate, defaults to 100Mhz
        num_samples: size of chunks, each sec contains 100 samples
        device: str: either of CPU or GPU

    Returns:
        signal: processed audio signal
        label: resulting label
    """
    labels_file: str
    waveform_file: str
    transformation: Any
    target_sample_rate: int
    num_samples: int
    device: str = config.DEVICE
    labels: Any = field(init=False)
    waveform_raw: Any = field(init=False)

    def __post_init__(self) -> None:
        self.labels = pd.read_parquet(self.labels_file)
        self.waveform_raw = pd.read_parquet(self.waveform_file)
        self.transformation = self.transformation.to(self.device)
        logger.info(f"Device in use: {self.device}")

    def __len__(self):
        """
        Returns: The len of processed data
        """
        return len(self.labels)

    def __getitem__(self, index):
        """
        Function to process every input
        Args:
            index: index of input to be processed
        Returns:
            audio feature corresponding to the applied transformation
        """
        label = self._get_audio_sample_label(index)
        signal = torch.tensor(np.array(self.waveform_raw.iloc[index]).reshape(1, 1000)).float()
        _sr = config.SAMPLE_RATE
        signal = signal.to(self.device)
        signal = self._resample_if_not_target_sample_rate(signal, _sr)
        signal = self._mix_down_if_not_mono(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label

    def _cut_if_necessary(self, signal):
        """
        Args:
            signal: Some audio signal
        Returns:
            truncated audio signal, to ensure all signals have the same len or num of samples
        """
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        """
        Pads signal if the length is shorter that the specified number of sample in the config file
        Args:
            signal: Some audio signal
        Returns: Right padded audio
        """
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_not_target_sample_rate(self, signal, sr):
        """
        Function to resample audio to sample target rate
        Args:
            signal: Some audio signal
            sr: Sample rate from config file
        Returns: resampled audio data
        """
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate).to(self.device)
            signal = resampler(signal)
        return signal

    @staticmethod
    def _mix_down_if_not_mono(signal):
        """
        Function to reduce audio to one/mono channel
        Args:
            signal: some audion data
        Returns: mono channel audio
        """
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_label(self, index):
        return self.labels.iloc[index, 0]

### Explaining `__len__` and `__getitem__`

* __len__: Returns the total number of samples in your dataset. PyTorch uses this method to know how many iterations to run during training.
* __getitem__: Retrieves a specific sample from the dataset using its index. This method returns the data and its corresponding label, which PyTorch uses during training to form mini-batches.

In [7]:
import torch

# Generate random data: 6 samples, each with 2 features
torch.manual_seed(0)  # For reproducibility
features = torch.rand(6, 2)

# Generate random target values (e.g., for a regression problem)
targets = torch.rand(6, 1)

print(f"Features:\n{features}")
print(f"\nTarget:\n{targets}")

Features:
tensor([[0.4963, 0.7682],
        [0.0885, 0.1320],
        [0.3074, 0.6341],
        [0.4901, 0.8964],
        [0.4556, 0.6323],
        [0.3489, 0.4017]])

Target:
tensor([[0.0223],
        [0.1689],
        [0.2939],
        [0.5185],
        [0.6977],
        [0.8000]])


In [8]:
from torch.utils.data import DataLoader

# Create an instance of the custom dataset
dataset = CustomDataset(data=features, targets=targets)

# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Example of iterating through the DataLoader
for idx, (batch_data, batch_labels) in enumerate(data_loader):
    print(f"Batch {idx+1}:\n========")
    print(f"Data:\n{batch_data}")
    print(f"Targets:\n{batch_labels}\n")

Batch 1:
Data:
tensor([[0.4556, 0.6323],
        [0.4963, 0.7682]])
Targets:
tensor([[0.6977],
        [0.0223]])

Batch 2:
Data:
tensor([[0.0885, 0.1320],
        [0.3489, 0.4017]])
Targets:
tensor([[0.1689],
        [0.8000]])

Batch 3:
Data:
tensor([[0.3074, 0.6341],
        [0.4901, 0.8964]])
Targets:
tensor([[0.2939],
        [0.5185]])



* `shuffle=True` ensures that the data is shuffled at the beginning of each epoch, which helps prevent the model from learning patterns based on the order of the data.

::::{dropdown} 🏋️ Exercise: What if we set `shuffle` to `False` and `batch_size` to 3?
::::{tip}
Check the Sample Size.
::::
::::